<a href="https://colab.research.google.com/github/QueenFi703/Wandb-/blob/main/Launch_Job_Creation_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# W&B Job Creation Quickstart

Learn how to create reusable jobs from W&B, which can be executed on different ML environments using Launch. This can be useful for:

- Model evaluation — define suites of tests to run on promising new model variants
- Retraining and fine tuning — repeat training or fine tuning on larger hardware or multiple nodes
- Scaled out hyperparameter sweeps — use larger machines, and more of them, to accelerate hyperparameter tuning.

Here, we'll go through [several ways](https://docs.wandb.ai/ref/cli/wandb-job/wandb-job-create) to a create a reusable job, using a FMNIST training job. (See more example jobs [here](https://github.com/wandb/launch-jobs/tree/main/jobs) or at https://wandb.ai/wandb/jobs/jobs )

In this quickstart we will:
1. Create a job from a **local directory**.
1. Create a job from a **Github repo**.
1. Create a job from a **training run**.
1. Create a **queue**.
1. Start an **agent**.
1. **Launch** the jobs!

From here, you'll be able to see it all working in the GUI and take the next step to scaling your ML work on powerful infrastructure.


# Set up W&B to get started

In [ ]:
# Install W&B
!pip install wandb > 0.17.0 -qqq

In [ ]:
# prompt: A slider using Jupyter widgets

import ipywidgets as widgets
from IPython.display import display

slider = widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(slider)

In [ ]:
# Log in to your free W&B Account
import wandb
wandb.login()

In [ ]:
WB_PROJECT = "job-creation-demo"
WB_ENTITY = "wandb-smle" # Add your entity

# 1) Create a Job from a **local directory**

Clone the example job repository.

In [ ]:
!git clone https://github.com/wandb/launch-jobs.git

Now run this command to create a job:




In [ ]:
! wandb job create -p $WB_PROJECT -e $WB_ENTITY -n "fashion-mnist-train" code launch-jobs/jobs/fashion_mnist_train -E 'job.py'

You now have a job created and ready to launch.  If you want to skip right to launching, go to step 4.  Otherwise, try two other ways to create jobs.

# 2) Create a job from a **Github repo**
When you run the command below, W&B will build a job from this public repo.

In [ ]:
! wandb job create -p $WB_PROJECT -e $WB_ENTITY -n "First_Git_Job" git https://github.com/wandb/launch-jobs/tree/main/jobs/fashion_mnist_train -E 'job.py'

# 3) Create a job from **existing training code**

Next, we're going to create a job by logging the code to W&B.  We'll create a tiny sample Python script, which will be used to create a W&B job using `log_code()`.

In [ ]:
demo_job = """
import wandb
import random

with wandb.init(project="fmnist_job_tutorial") as run:
  run.log({"eval_metric": random.random()})
  print("This job could run an Optuna Study automatically, for example.")
  run.log_code()
"""

with open('demo_job.py', 'w+') as file:
  file.write(demo_job)

requirements = """
wandb
random
"""

with open('requirements.txt', 'w+') as file:
  file.write(requirements)

Now run the training code.

In [ ]:
!python demo_job.py

# 4) Create a Launch Queue

Now let's create a queue to run these Jobs on. Queues are our interface into ML infrastructure — W&B supports Kubernetes, Sagemaker, and Docker queues (the latter being most useful for individual workstations).


In [ ]:
import json
config ={"label":"tutorial-test"}
api = wandb.Api()
queue = api.create_run_queue('tutorial-run-queue', 'local-container', prioritization_mode='V0', entity=WB_ENTITY, config=config)

# 5) Start an Agent

The agent will pull down Jobs from the queue to run in Docker containers — install and start [Docker](https://www.docker.com/products/docker-desktop/) to use the agent on your laptop.


## 💻 `in your laptop terminal`
```shell
pip install wandb --upgrade
wandb login
wandb launch-agent -q tutorial-run-queue
```

# 6) Launch the job

Now we're all set to launch the job.  Run the command below, then look over at your terminal to see the job in progress.

In [ ]:
! wandb launch -j "$WB_ENTITY/$WB_PROJECT/fashion-mnist-train:latest" -q tutorial-run-queue

# 🎉 You did it! 🎉

You registered a job and ran it using Launch! If you go to your project, you'll see a new run corresponding to the job you just ran.

To run a hyperparameter sweep on your queue--with condition configuration and multi-objective optimization--jump over to [this notebook](https://colab.research.google.com/drive/1WxLKaJlltThgZyhc7dcZhDQ6cjVQDfil#scrollTo=sHcr30CKybN7).  
